In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [40]:
from ultralytics import YOLO
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def apply_circular_mosaic(image, x, y, w, h, mosaic_size=4):
    """
    얼굴 영역을 원형 모자이크 처리하는 함수
    """
    face_region = image[y:y+h, x:x+w].copy()
    h, w = face_region.shape[:2]

    # 모자이크 적용
    for i in range(0, h, mosaic_size):
        for j in range(0, w, mosaic_size):
            y1 = i
            y2 = min(i + mosaic_size, h)
            x1 = j
            x2 = min(j + mosaic_size, w)
            color = cv2.mean(face_region[y1:y2, x1:x2])[:3]
            face_region[y1:y2, x1:x2] = color

    # 원형 마스크 생성
    mask = np.zeros((h, w), dtype=np.uint8)
    center = (w // 2, h // 2)
    radius = min(w, h) // 2
    cv2.circle(mask, center, radius, 255, -1)

    # 원형 모자이크 적용
    circular_mosaic = cv2.bitwise_and(face_region, face_region, mask=mask)
    face_region_bg = cv2.bitwise_and(image[y:y+h, x:x+w], image[y:y+h, x:x+w], mask=cv2.bitwise_not(mask))
    combined = cv2.addWeighted(face_region_bg, 1, circular_mosaic, 1, 0)

    image[y:y+h, x:x+w] = combined
    return image

def process_frames(frames, model):
    """
    여러 프레임을 처리하여 원형 모자이크를 적용하는 함수
    """
    results = model(frames)
    processed_frames = []
    for i, frame in enumerate(frames):
        for result in results[i].boxes:
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # 좌표를 정수로 변환
            w, h = x2 - x1, y2 - y1
            frame = apply_circular_mosaic(frame, x1, y1, w, h)
        processed_frames.append(frame)
    return processed_frames

# 학습된 모델 로드 (GPU 사용)
model = YOLO('/content/drive/MyDrive/test/best.pt')
model.to('cuda')

# 비디오 파일 로드
video_path = '/content/drive/MyDrive/test/test_video.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("비디오 파일을 열 수 없습니다.")
    exit()

# 비디오 파일 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/drive/MyDrive/test/Mosaic_output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

batch_size = 8  # 배치 크기 설정
frames = []
executor = ThreadPoolExecutor(max_workers=4)

# 비디오 읽기 및 처리 비동기화
def read_frames(cap, batch_size):
    while True:
        batch = []
        for _ in range(batch_size):
            ret, frame = cap.read()
            if not ret:
                return
            batch.append(frame)
        if len(batch) == 0:
            break
        yield batch

# 메인 처리 루프
futures = []
frame_index = 0
for batch in read_frames(cap, batch_size):
    future = executor.submit(process_frames, batch, model)
    futures.append((future, frame_index))
    frame_index += len(batch)

# 모든 작업 완료 후 결과 쓰기
futures.sort(key=lambda x: x[1])  # frame_index 순서로 정렬
for future, _ in futures:
    processed_frames = future.result()
    for frame in processed_frames:
        out.write(frame)

# 자원 해제
cap.release()
out.release()
executor.shutdown()
print("비디오 저장이 완료되었습니다.")





0: 384x640 4 faces, 24.7ms
1: 384x640 4 faces, 24.7ms
2: 384x640 4 faces, 24.7ms
3: 384x640 4 faces, 24.7ms
4: 384x640 4 faces, 24.7ms
5: 384x640 4 faces, 24.7ms
6: 384x640 4 faces, 24.7ms
7: 384x640 4 faces, 24.7ms
Speed: 3.0ms preprocess, 24.7ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
0: 384x640 4 faces, 4.8ms
1: 384x640 4 faces, 4.8ms
2: 384x640 4 faces, 4.8ms
3: 384x640 4 faces, 4.8ms
4: 384x640 4 faces, 4.8ms
5: 384x640 4 faces, 4.8ms
6: 384x640 4 faces, 4.8ms
7: 384x640 4 faces, 4.8ms
Speed: 6.9ms preprocess, 4.8ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

0: 384x640 4 faces, 14.2ms
1: 384x640 4 faces, 14.2ms
2: 384x640 4 faces, 14.2ms
3: 384x640 4 faces, 14.2ms
4: 384x640 3 faces, 14.2ms
5: 38

In [41]:
from ultralytics import YOLO
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def draw_face_detection(image, x, y, w, h):
    """
    얼굴 영역에 사각형을 그리는 함수
    """
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return image

def process_frames(frames, model):
    """
    여러 프레임을 처리하여 얼굴 감지를 적용하는 함수
    """
    results = model(frames)
    processed_frames = []
    for i, frame in enumerate(frames):
        for result in results[i].boxes:
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # 좌표를 정수로 변환
            w, h = x2 - x1, y2 - y1
            frame = draw_face_detection(frame, x1, y1, w, h)
        processed_frames.append(frame)
    return processed_frames

# 학습된 모델 로드 (GPU 사용)
model = YOLO('/content/drive/MyDrive/test/best.pt')
model.to('cuda')

# 비디오 파일 로드
video_path = '/content/drive/MyDrive/test/test_video.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("비디오 파일을 열 수 없습니다.")
    exit()

# 비디오 파일 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/drive/MyDrive/test/output_detection.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

batch_size = 8  # 배치 크기 설정
frames = []
executor = ThreadPoolExecutor(max_workers=4)

# 비디오 읽기 및 처리 비동기화
def read_frames(cap, batch_size):
    while True:
        batch = []
        for _ in range(batch_size):
            ret, frame = cap.read()
            if not ret:
                return
            batch.append(frame)
        if len(batch) == 0:
            break
        yield batch

# 메인 처리 루프
futures = []
frame_index = 0
for batch in read_frames(cap, batch_size):
    future = executor.submit(process_frames, batch, model)
    futures.append((future, frame_index))
    frame_index += len(batch)

# 모든 작업 완료 후 결과 쓰기
futures.sort(key=lambda x: x[1])  # frame_index 순서로 정렬
for future, _ in futures:
    processed_frames = future.result()
    for frame in processed_frames:
        out.write(frame)

# 자원 해제
cap.release()
out.release()
executor.shutdown()
print("비디오 저장이 완료되었습니다.")





0: 384x640 4 faces, 46.9ms
1: 384x640 4 faces, 46.9ms
2: 384x640 4 faces, 46.9ms
3: 384x640 4 faces, 46.9ms
4: 384x640 4 faces, 46.9ms
5: 384x640 4 faces, 46.9ms
6: 384x640 4 faces, 46.9ms
7: 384x640 4 faces, 46.9ms
Speed: 3.7ms preprocess, 46.9ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 faces, 6.2ms
1: 384x640 4 faces, 6.2ms
2: 384x640 4 faces, 6.2ms
3: 384x640 4 faces, 6.2ms
4: 384x640 4 faces, 6.2ms
5: 384x640 4 faces, 6.2ms
6: 384x640 4 faces, 6.2ms
7: 384x640 4 faces, 6.2ms
Speed: 7.5ms preprocess, 6.2ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
0: 384x640 4 faces, 4.1ms
1: 384x640 4 faces, 4.1ms
2: 384x640 4 faces, 4.1ms
3: 384x640 4 faces, 4.1ms
4: 384x640 4 faces, 4.1ms
5: 384x640 4 faces, 4.1ms
6: 384x640 4 faces, 4.1ms
7: 384x640 4 faces, 4.1ms
Speed: 7.8m